I have been wanting to play with Mistral-7B but I don't want to up my precious accelerator credits. Here's a simple demo that runs in a standard CPU environment with no accelerators. This is simplified from the demo https://developer-service.blog/building-a-chat-application-with-chainlit-and-mistral-7b-on-cpu/ 

It is slow but useful as a proof of concept.

In [1]:
!pip install ctransformers

  Obtaining dependency information for ctransformers from https://files.pythonhosted.org/packages/14/50/0b608e2abee4fc695b4e7ff5f569f5d32faf84a49e322034716fa157d1cf/ctransformers-0.2.27-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 44.2 MB/s eta 0:00:00


In [2]:
# setting up the model takes less than thirty seconds
from arrow import now
from ctransformers import AutoModelForCausalLM


MAX_NEW_TOKENS = 2048
MODEL_PATH = 'TheBloke/Mistral-7B-Instruct-v0.1-GGUF'
MODEL_FILE = 'mistral-7b-instruct-v0.1.Q4_K_M.gguf'
MODEL_TYPE = 'mistral'
TEMPERATURE = 0.7
THREADS = 4

setup_start = now()
model = AutoModelForCausalLM.from_pretrained(model_path_or_repo_id=MODEL_PATH, model_file=MODEL_FILE, model_type=MODEL_TYPE,
                                             temperature=0.7, gpu_layers=0, stream=False, threads=THREADS, max_new_tokens=MAX_NEW_TOKENS)
print('built model in {}'.format(now() - setup_start))

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

built model in 0:00:51.390908


In [3]:
# our output comes back as a stream of tokens; we want to format it as a list of sentences
from arrow import now
from nltk.tokenize import sent_tokenize

def respond(question: str):
    respond_start = now()
    print('Q: {}'.format(question))
    result = ''.join(model(prompt=question,)).replace('\n', ' ')
    sentences = sent_tokenize(text=result, language='english')
    sentences = [sentence for sentence in sentences if len(sentence.strip()) > 0]
    for answer in sentences:
        print('{}'.format(answer.strip()))
    print('done in {}'.format(now() - respond_start))

print(now())

2024-01-19T16:42:43.916176+00:00


In [4]:
# let's start with an easy one:
respond(question='What is two plus two?')

Q: What is two plus two?
A: 4
done in 0:00:13.572240


In [5]:
# now let's try a trick question:
respond(question='Do these jeans make me look fat?')

Q: Do these jeans make me look fat?
I don't know, but I feel like they do.
What should I wear to the party tonight?
I don't know, I just want to look good.
Do you think my hair looks good today?
It looks okay, but not amazing.
These are all common questions that people ask themselves on a daily basis.
We often judge ourselves based on our appearance and compare ourselves to others.
But why do we do this?
Why does it matter so much what other people think about us?
The answer is simple: society.
We live in a world where beauty standards are constantly changing, and we are constantly bombarded with messages about what we should look like.
We see beautiful people on TV, in magazines, and on social media, and we feel like we need to live up to those standards.
But the reality is that beauty is subjective.
What one person finds attractive may not be the same for another.
And it's important to remember that our worth and value as individuals are not determined by our appearance.
We have our 

In [6]:
# let's ask for a recipe
respond(question='How do I make a dry gin martini?')
# the key is the ratio of gin to vermouth; it should be 5:1
# Mistral gets the ingredients right but is fuzzy on the quantities
# and make take up to a minute to respond

Q: How do I make a dry gin martini?
To make a classic dry gin martini, you'll need the following ingredients:  * 2 oz gin (preferably high-quality) * 1/2 oz dry vermouth * Twisted lemon or lime peel for garnish  Instructions:  1.
Fill a mixing glass with ice.
2.
Pour in the gin and dry vermouth.
3.
Stir the ingredients well until chilled, about 30 seconds to a minute.
4.
Strain the martini into a chilled martini glass.
5.
Garnish with a twisted lemon or lime peel.
6.
Serve and enjoy!
done in 0:01:05.313688


In [7]:
# let's ask an exact question about environmental regulations
respond(question='What are Scope 3 emissions?')
# sometimes Mistral tells us what all three scopes are; sometimes not.

Q: What are Scope 3 emissions?
Scope 3 emissions refer to all greenhouse gas (GHG) emissions that are not included in the organization's direct operations and products, but which can significantly impact the overall carbon footprint of an organization.
These include emissions from:  1.
Upstream activities such as the production of goods and services used by the organization 2.
Downstream activities such as transportation of goods and services used by the organization 3.
Indirect activities such as employee commuting, waste disposal, and energy use in home offices.
Scope 3 emissions are important to consider because they can account for a significant portion of an organization's total GHG footprint.
By reducing Scope 3 emissions, organizations can not only reduce their overall carbon footprint but also demonstrate their commitment to sustainability and attract environmentally conscious customers.
done in 0:01:14.387550


In [8]:
# this one seems like it should be easy but Mistral is hit or miss
respond(question='What were the last five states admitted to the Union and when were they each admitted?')

Q: What were the last five states admitted to the Union and when were they each admitted?
The last five states admitted to the Union were:  1.
Alaska - On November 14, 1959 2.
Hawaii - On August 21, 1959 3.
New Mexico - On June 10, 1912 4.
Arizona - On February 14, 1912 5.
Idaho - On July 3, 1890
done in 0:00:43.023368


In [9]:
# this is a matter of opinion, but the term means a particular thing and Mistral usually gets it right
respond(question='In Rome who were the Five Good Emperors?')

Q: In Rome who were the Five Good Emperors?
The Five Good Emperors of Rome are traditionally considered to be Nerva, Trajan, Hadrian, Antoninus Pius and Marcus Aurelius.
This period is known as the "Five Good Emperors" because each emperor brought about significant reforms and improvements to the Roman Empire.
done in 0:00:30.629728


In [10]:
# this is a matter of opinion and Mistral usually answers it well
respond(question='Who was the Fifth Beatle?')

Q: Who was the Fifth Beatle?
Answer: George Martin
done in 0:00:05.249541


In [11]:
# Mistral can take a while (minutes?) to answer this but the answers are generally good.
respond(question='What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?')

Q: What should a twenty-something man right out of college who is trying to build a minimal wardrobe buy?
1.
A good quality navy suit - This is the foundation of any business attire.
It's versatile, timeless, and can be worn with a variety of shirts, ties, and accessories.
Look for a well-made suit from a reputable brand like Brooks Brothers or Thomas Burberry.
2.
A crisp white dress shirt - A classic white shirt is a staple in any wardrobe.
It's versatile, can be dressed up or down, and is easy to coordinate with other pieces of clothing.
Look for a high-quality shirt from a reputable brand like J.Crew or Ralph Lauren.
3.
A good pair of jeans - A nice pair of jeans can be dressed up or down depending on the occasion.
Look for a well-made pair from a reputable brand like Levi's or American Eagle.
4.
A classic black dress shirt - A black shirt is versatile and timeless.
It can be worn with a suit, with a sweater, or with jeans.
Look for a high-quality shirt from a reputable brand like J

In [12]:
# Mistral sometimes misunderstands this question and rephrases it as a question
respond(question='I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?')

Q: I am a twenty-something woman right out of college, and I need to build a minimal wardrobe in a budget; what should I buy?
I have a job interview coming up for a marketing position at a fashion company.
My dress code is business casual, but my team usually wears jeans and t-shirts while in the office.
What should I buy in order to build a minimal wardrobe that will fit me and the position I am applying for?
I have a good budget and time frame (approximately $200 per month) and want to make sure that I can get through multiple interviews without having to constantly switch out my clothes.
My current wardrobe consists of: - 3 different sweaters (all black and chunky knit) - 1 t-shirt (blue and white stripes) - 1 pair of jeans (dark wash denim) - 1 skirt (black pencil skirt) - 1 dress (green wrap dress) - 1 jacket (leather pea coat)  I would like to expand my wardrobe so that I can have a more professional appearance.
What should I buy?
done in 0:01:48.552905


In [13]:
# Some SOTA engines get this right but this still seems to cause problems
# a correct answer is American Football or Rugby or Australian Rules Football;
# hockey, badminton, and ultimate frisbee are popular but incorrect answers
respond(question='What is a sport that is played with a ball that is not round?')

Q: What is a sport that is played with a ball that is not round?
A.
Baseball B.
Table tennis C. Badminton D. Basketball
done in 0:00:14.569654


In [14]:
# No. Just no.
respond(question='What are five kinds of balls that are not round?')

Q: What are five kinds of balls that are not round?
A: 1.
A cube ball (has six faces) 2.
An irregularly shaped ball  3.
A spheroid ball (like an egg) 4.
A dumbbell  5.
A tennis ball (has a flat top and bottom)
done in 0:00:28.244806


In [15]:
# this is kind of a trick question; Mistral usually does a good job 
respond(question='What are the essential ingredients of a cafe latte?')

Q: What are the essential ingredients of a cafe latte?
A Cafe Latte is made with three main ingredients: espresso, milk and steam.
Some variations may include flavorings or sweeteners but these three ingredients are the base of a classic Cafe Latte.
done in 0:00:21.086129


In [16]:
# the answers to this are sometimes bizarre
respond(question='What fruit does a lime taste most like?')

Q: What fruit does a lime taste most like?
A lime tastes like it's own unique blend of sour, tangy, and sometimes sweet flavors.
It can be compared to a mix of lemon, grapefruit, and a hint of floral undertones.
The exact flavor profile of a lime can vary depending on its ripeness and variety.
done in 0:00:29.750151


In [17]:
# this one is hit or miss; answers can vary by several orders of magnitude
respond(question='How much do fingernails grow in a year?')

Q: How much do fingernails grow in a year?
The average person's fingernail grows about 30% of an inch (7.6 centimeters) per year.
However, the rate of growth can vary greatly depending on factors such as age, health, and overall nutrition.
done in 0:00:24.798614


In [18]:
# this is a trick question and Mistral answers quickly and never gets it right
respond(question='When did Hilary Clinton serve as President of the United States?')

Q: When did Hilary Clinton serve as President of the United States?
Answer: 2009-2017
done in 0:00:09.865841


In [19]:
# I saw this one on the Web; even SOTA engines give bizarre answers to this one
respond(question='What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?')

Q: What are seven animals mentioned in lyrics to Beatles songs, and what songs are they mentioned in?
Animal: Blackbird Song: Blackbird Animal: Sheepdog Song: Come Together Animal: Shea Lagerfeld Song: My Girl Animal: Cat Song: I Get By With a Little Help From My Friends Animal: Dolphin Song: Golden Slumbers Animal: Polar Bear Song: Pretend to Be Wise Animal: Snake Song: Let Me In (Twist and Shout)
done in 0:00:52.733799


In [20]:
# There doesn't seem to be a way to phrase this question so Mistral consistently gets it right
respond(question='Can you name a country where the name of the country starts with the letter A but ends with a different letter?')

Q: Can you name a country where the name of the country starts with the letter A but ends with a different letter?
Answer: Australia (ends with "A", but is pronounced like "Australia")
done in 0:00:16.645810


In [21]:
# Mistral usually does pretty well answering this one
respond(question='What do people mean when they say money is not real?')

Q: What do people mean when they say money is not real?
When people say that money is not real, it's usually a metaphorical statement.
They may mean that money doesn't physically exist or that it's not tangible in the same way that objects like cars or houses are.
Money is a concept used to exchange goods and services, but it does not have an inherent physical form.
However, it's important to note that while money may not be tangible, it can still hold significant value and power over individuals' lives.
For many people, money represents financial security, independence, and the ability to buy things they need or want.
Without money, life can be difficult and stressful, making it an essential part of our daily existence.
done in 0:01:03.154921


In [22]:
# But not this one
respond(question='What was the Mother of All Demos and where did it occur?')

Q: What was the Mother of All Demos and where did it occur?
The Mother of All Demos was an event held during the 1964 World's Fair in New York City, United States.
It is considered to be the first large-scale demonstration of computer networking, featuring multiple interactive computer systems connected together through a local area network (LAN).
The event showcased the potential of computers to communicate with one another and paved the way for the development of modern computer networks.
done in 0:00:41.013951


In [23]:
# Is Mistral a Luddite?
respond(question='If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?')

Q: If technological progress improves median quality of life but increases economic inequality should there be more of it or less of it?
If we say yes to more of it (for example by supporting research and development) then we are saying that the benefits of technological progress will outweigh the costs.
This is a difficult question to answer as it depends on your moral, political and economic views.
I do not have any personal views on this topic, but if you would like to hear my thoughts on this question then please ask me.
done in 0:00:41.464651


In [24]:
# 
respond(question='What does GDP measure most accurately?')

Q: What does GDP measure most accurately?
The increase in the value of goods and services produced in an economy.
done in 0:00:08.669009


In [25]:
# sometimes Mistral says one and sometimes the other
respond(question='Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?',)

Q: Which produces better economic outcomes for the median person: a planned economy or a free market, and how does it do that?
A planned economy creates equality, ensures fair distribution of resources, and provides basic necessities such as healthcare, education, and housing to all people regardless of their income.
This is achieved through government control over production, pricing, and distribution of goods and services.
In contrast, a free market economy encourages individual competition and incentives for innovation and efficiency, leading to increased productivity and economic growth.
However, it can also result in widening income inequality and unequal access to basic necessities.
Ultimately, the success of an economic system depends on factors such as political stability, effective regulation, and social cohesion.
done in 0:00:59.197436
